In [1]:
%load_ext autoreload
%autoreload 2
#imports
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.utils import serve_utils
from aiflows.utils import colink_utils
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
import sys
sys.path.append("..")
from utils import compile_and_writefile
from aiflows import flow_verse
import json

#Specify path of your flow modules
FLOW_MODULES_PATH = "./"

/Users/nicolasbaldwin/opt/miniconda3/envs/coflows/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Connect to the CoLink Server

In [2]:
cl = colink_utils.start_colink_server()

started server on 0.0.0.0:17921
started server on 0.0.0.0:17986


## 2. ChatFlow

### 2.1 Importing ChatFlow From the FlowVerse

In [3]:
#import the ChatFlowModule from the flowverse on hugging face
# link: https://huggingface.co/aiflows/ChatFlowModule/tree/coflows
dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
]
#Dowload flow modules in "./flow_modules"
flow_verse.sync_dependencies(dependencies)


[2024-03-18 10:18:34,864][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules...
[2024-03-18 10:18:35,255][aiflows.flow_verse.loading:612][WARNING] - [<interactive>] aiflows/ChatFlowModule:coflows's commit hash has changed from 0fb662fd9b563a160e3141f42f22da226dd39879 to 11ac7906787a4e149b39fe5cf42e1b5d47a0288c, as synced module is not modified, the newest commit regarding aiflows/ChatFlowModule:coflows will be fetched


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 2701.74it/s]


[2024-03-18 10:18:36,267][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/Users/nicolasbaldwin/Documents/OneDrive/EPFL/DLAB/aiflow-colink/aiflows/AMLD/ChatFlow/flow_modules/aiflows/ChatFlowModule']

In [4]:
## Print the configuration
default_cfg = read_yaml_file("flow_modules/aiflows/ChatFlowModule/ChatAtomicFlow.yaml")
print(json.dumps(default_cfg, indent=4))


{
    "_target_": "flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config",
    "name": "ChatAtomicFlow",
    "description": "Flow which uses as tool an LLM though an API",
    "enable_cache": true,
    "n_api_retries": 6,
    "wait_time_between_retries": 20,
    "system_name": "system",
    "user_name": "user",
    "assistant_name": "assistant",
    "backend": {
        "_target_": "aiflows.backends.llm_lite.LiteLLMBackend",
        "api_infos": "???",
        "model_name": "gpt-3.5-turbo",
        "n": 1,
        "max_tokens": 2000,
        "temperature": 0.3,
        "top_p": 0.2,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stream": true
    },
    "system_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt"
    },
    "init_human_message_prompt_template": {
        "_target_": "aiflows.prompt_template.JinjaPrompt"
    },
    "human_message_prompt_template": {
        "_target_": "aiflows.prompt

### 2.2 Serving & Getting and Instance of the ChatFlow

#### 2.2.1 Serving the ChatFlow

In [5]:
serve_utils.serve_flow(
    cl = cl,
    flow_type="ChatFlow",
    default_config=default_cfg
)

Started serving at flows:ChatFlow.


True

#### 2.2.1 Getting an Instance of the ChatFlow

In [ ]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl, dispatch_point="coflows_dispatch", flow_modules_base_path=FLOW_MODULES_PATH)

# Get an instance of the flow
proxy_reverse_number_flow = serve_utils.recursive_mount(
    cl=cl,
    client_id="local",
    flow_type="ReverseNumberAtomicFlow_served", 
    config_overrides=None,
    initial_state=None,
    dispatch_point_override=None,
)